<a href="https://colab.research.google.com/github/SarahLares/KPMG_virtual_intership/blob/master/Data_Insightsy_Assessment_KPMG_Virtual_intership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [101]:
df_Cust_add = pd.read_csv('CustomerAddress.csv')
df_Cust_dem = pd.read_csv('CustomerDemographic.csv')
df_New_cust = pd.read_csv('NewCostumerList.csv')
df_Transactions = pd.read_csv('Transactions.csv')

> As seen in the previous task, there were not the same number of customers in the data sets **CustomerDemographic** and **CustomerAddress** so, only the customer_id that are in both data sets will be used and both sets will be united in one single Dataframe







In [102]:
all_cust = pd.merge(left=df_Cust_add,right=df_Cust_dem, left_on='customer_id', right_on='customer_id')

In [103]:
all_cust.shape

(3996, 18)

In [104]:
all_cust.head()

,customer_id,address,postcode,state,country,property_valuation,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,060 Morning Avenue,2016,New South Wales,Australia,10,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,4,0 Holy Cross Court,4211,QLD,Australia,9,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0
4,6,9 Oakridge Court,3216,VIC,Australia,9,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,ðµ ð ð ð,Yes,13.0


In [105]:
print('Missing Values in the Master DF \n\n{}'.format(all_cust.isnull().sum()))

Missing Values in the Master DF 

customer_id                              0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              504
job_industry_category                  655
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64




> We will delete the first_name  , last_name, country and default columns from the dataset.



In [106]:
all_cust = all_cust.drop(columns=['first_name','last_name','country','default'])

In [107]:
all_cust['gender'].value_counts()

Female    2035
Male      1870
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64



> We can see that we don't know the gender of 2% of customers, so we will delete this rows.



In [108]:
all_cust = all_cust.drop(labels=all_cust['gender'].loc[all_cust['gender']=='U'].index)

In [109]:
print('Missing Values in the Master DF \n\n{}'.format(all_cust.isnull().sum()))

Missing Values in the Master DF 

customer_id                              0
address                                  0
postcode                                 0
state                                    0
property_valuation                       0
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                      0
job_title                              495
job_industry_category                  655
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
dtype: int64




> we can see that now the data still have missing values in 2 columns. 

> First, let's proceed to write the column 'gender' in a consistent way.

In [115]:
all_cust['gender'].replace(to_replace=['Male','Female','Femal'], value=['M','F','F'],inplace=True)

In [111]:

all_cust['owns_car'].replace(to_replace=['No','Yes'], value=['N','Y'],inplace=True)

In [112]:
all_cust['state'].replace(to_replace=['New South Wales','Victoria'], value=['NSW','VIC'],inplace=True)

In [113]:
all_cust['DOB']= all_cust['DOB'].map(lambda x: 2020-int(x[:4]))
all_cust.rename(columns={'DOB':'age'})

,customer_id,address,postcode,state,property_valuation,gender,past_3_years_bike_related_purchases,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,060 Morning Avenue,2016,NSW,10,1,93,67,Executive Secretary,Health,Mass Customer,N,Y,11.0
1,2,6 Meadow Vale Court,2153,NSW,10,0,81,40,Administrative Officer,Financial Services,Mass Customer,N,Y,16.0
2,4,0 Holy Cross Court,4211,QLD,9,0,33,59,NaN,IT,Mass Customer,N,N,7.0
3,5,17979 Del Mar Point,2448,NSW,4,1,56,43,Senior Editor,NaN,Affluent Customer,N,Y,8.0
4,6,9 Oakridge Court,3216,VIC,9,0,35,54,NaN,Retail,High Net Worth,N,Y,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,3995,613 Erie Lane,2088,NSW,12,0,93,45,Staff Scientist,Manufacturing,Mass Customer,N,Y,14.0
3991,3996,0 Transport Center,3977,VIC,6,1,8,45,VP Product Management,Health,Mass Customer,N,N,19.0
3992,3997,4 Dovetail Crossing,2350,NSW,2,1,87,19,Statistician II,Manufacturing,High Net Worth,N,Y,1.0
3994,3999,1482 Hauk Trail,3064,VIC,3,0,11,47,NaN,Manufacturing,Affluent Customer,N,Y,10.0


In [116]:
all_cust.head()

,customer_id,address,postcode,state,property_valuation,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,060 Morning Avenue,2016,NSW,10,F,93,67,Executive Secretary,Health,Mass Customer,N,Y,11.0
1,2,6 Meadow Vale Court,2153,NSW,10,M,81,40,Administrative Officer,Financial Services,Mass Customer,N,Y,16.0
2,4,0 Holy Cross Court,4211,QLD,9,M,33,59,NaN,IT,Mass Customer,N,N,7.0
3,5,17979 Del Mar Point,2448,NSW,4,F,56,43,Senior Editor,NaN,Affluent Customer,N,Y,8.0
4,6,9 Oakridge Court,3216,VIC,9,M,35,54,NaN,Retail,High Net Worth,N,Y,13.0


In [ ]:
trans_users = []
for cust in all_cust['customer_id']:
  tran = 0
  tran = df_Transactions.loc[df_Transactions['customer_id']==cust].shape[0]
  trans_users.append(tran)


We calculate the number of transaction for each **user_id** in the last 3 months and this is a new column in the data set

In [ ]:
tran = pd.DataFrame(trans_users, columns=['Transactions_last_3_months'])

In [ ]:
all_cust = pd.concat([all_cust,tran], axis=1)

In [ ]:
all_cust['Transactions_last_3_months'].describe()

count    3996.000000
mean        4.996997
std         2.890404
min         0.000000
25%         3.000000
50%         5.000000
75%         7.000000
max        14.000000
Name: Transactions_last_3_months, dtype: float64

We can see that the mean of transaction in last 3 months for all teh customer is around 5 and 50% of clients made this number of transactions. And approximately 30% of customers made 7 transactions. 

In [ ]:
print("Percentage of customers who made 7 or more transactions in the last 3 months {:.5}".format(all_cust.loc[all_cust['Transactions_last_3_months']>= 7].shape[0]*100/3996))

Percentage of customers who made 7 or more transactions in the last 3 months 30.28
